In [15]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from geopandas.tools import sjoin
import pandas as pd
import networkx as nx

import osm_extract

# read tunnel data as example
osm_df = gpd.read_file(r'C:/Joanne_PSRC/temp_data/final_network2.shp').to_crs(2285)
nodes = gpd.read_file(r'C:/Joanne_PSRC/temp_data/all_nodes.shp')
# islands = gpd.read_file(r'C:/Joanne_PSRC/temp_data/islands.shp').to_crs(2285)


In [1]:
import geopandas_osm.osm


In [16]:
# psrc_buffer = gpd.read_file(r'C:/Joanne_PSRC/temp_data/PS_buffer.shp').to_crs(2285)
# islands_wID = gpd.read_file(r'C:/Joanne_PSRC/temp_data/islands_wID.shp').to_crs(2285)
osm_gdf = gpd.read_file(r'W:/gis/projects/OSM/scripts/data/osm_extract.shp').to_crs(2285)
highway_types = ['trunk', 'trunk_link', 'motorway', 'motorway_link',
 'primary', 'primary_link', 'residential', 'road', 'secondary',
 'service', 'tertiary', 'tertiary_link', 'secondary_link',
 'cycleway', 'footway', 'pedestrian', 'living_street', 'unclassified',
 'steps']

In [17]:
split_net = osm_extract.generate_network_topology(osm_gdf, highway_types)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


start splitting tunnels


C:\Joanne_PSRC\travel_models\all_streets_network\osm_extract.py:80: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  geom = [i for i in un]


links that are not joined back to the original roads
Empty GeoDataFrame
Columns: [id, geometry, highway, name, bridge, tunnel, oneway, road_id]
Index: []
list of all links that are duplicated in the process
[]
start splitting bridges


C:\Joanne_PSRC\travel_models\all_streets_network\osm_extract.py:80: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  geom = [i for i in un]


links that are not joined back to the original roads
Empty GeoDataFrame
Columns: [id, geometry, highway, name, bridge, tunnel, oneway, road_id]
Index: []
list of all links that are duplicated in the process
['b-1155', 'b-1156']
start splitting roads


C:\Joanne_PSRC\travel_models\all_streets_network\osm_extract.py:80: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  geom = [i for i in un]


links that are not joined back to the original roads
              id                                           geometry highway  \
514893  r-514894  LINESTRING (1313215.271 233076.920, 1313183.67...     NaN   
767249  r-767250  LINESTRING (1283153.024 327401.868, 1283199.79...     NaN   

       name bridge tunnel oneway  road_id  
514893  NaN    NaN    NaN    NaN      NaN  
767249  NaN    NaN    NaN    NaN      NaN  
list of all links that are duplicated in the process
['r-71681', 'r-150200', 'r-150201', 'r-202027', 'r-266642', 'r-266643', 'r-266645', 'r-291919', 'r-331930', 'r-331931', 'r-339480', 'r-339481', 'r-339482', 'r-350103', 'r-351986', 'r-352513', 'r-362753', 'r-365876', 'r-402995', 'r-414554', 'r-415526', 'r-415533', 'r-415539', 'r-415544', 'r-431495', 'r-431499', 'r-457002', 'r-457003', 'r-463572', 'r-463573', 'r-463574', 'r-470953', 'r-479525', 'r-480752', 'r-480753', 'r-480754', 'r-486720', 'r-486721', 'r-486722', 'r-486723', 'r-486724', 'r-486725', 'r-486726', 'r-48672

In [18]:
split_net.columns

Index(['id', 'geometry', 'highway', 'name', 'bridge', 'tunnel', 'oneway',
       'road_id', 'link_id'],
      dtype='object')

In [ ]:
x_graph = nx.Graph()
graph = nx.from_pandas_edgelist(gdf, 'i_node', 'j_node',
                                ['id', 'geometry', 'highway', 'name', 'bridge', 'tunnel', 'oneway', 'road_id',
                                 'link_id', 'i_node', 'j_node'], x_graph)
# find all sub_networks
sep_graph = [graph.subgraph(c).copy() for c in sorted(nx.connected_components(graph), key=len, reverse=True)]

# combine all edges other than the main network
df_list = []
for i in range(1, len(sep_graph)):  # not including main network
    df = nx.to_pandas_edgelist(sep_graph[i])
    df['sub_net_id'] = i  # name each sub-network
    df_list.append(df)
all_disconnected = pd.concat(df_list, ignore_index=True)

# create a gdf out of the disconnected network and clip with island shp
gdf_disconnect = gpd.GeoDataFrame(all_disconnected, geometry='geometry').set_crs(crs)

# keep only networks on islands: clip disconnected networks using islands
islands = gpd.read_file(Path(islands)).to_crs(crs)
island_network = gpd.sjoin(gdf_disconnect, islands[['countyname', 'geoid20', 'island_id', 'geometry']], how="inner", op='within')
# keep largest network in islands
island_network = island_network.loc[island_network.groupby(['island_id'])['sub_net_id'].apply(lambda x: x==min(x))]

# gdf of main network
df_final_network = nx.to_pandas_edgelist(sep_graph[0]).append(island_network)
final_network = gpd.GeoDataFrame(df_final_network, geometry='geometry').set_crs(crs)


# remove sub-networks inside islands

In [3]:
x_graph = nx.Graph()
graph = nx.from_pandas_edgelist(osm_df, 'i_node', 'j_node',
                                ['id', 'geometry', 'highway', 'name', 'bridge', 'tunnel', 'oneway', 'road_id',
                                 'link_id', 'i_node', 'j_node'], x_graph)


In [10]:
# find all sub_networks
sep_graph = [graph.subgraph(c).copy() for c in sorted(nx.connected_components(graph), key=len, reverse=True)]

# combine all edges other than the main network
df_list = []
for i in range(1, len(sep_graph)): # not including main network
    df = nx.to_pandas_edgelist(sep_graph[i])
    df['sub_net_id'] = i

    df_list.append(df)
all_disconnected = pd.concat(df_list, ignore_index=True)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [ ]:
# create a gdf out of the disconnected network and clip with island shp
gdf_disconnect = gpd.GeoDataFrame(all_disconnected, geometry='geometry').set_crs(2285)

In [33]:
# keep only networks on islands: clip disconnected networks using islands
# islands = gpd.read_file(Path(islands)).to_crs(crs)
island_network = gpd.sjoin(gdf_disconnect, islands_wID[['countyname', 'geoid20', 'island_id', 'geometry']], how="inner", op='within')
# keep largest network in islands
island_network = island_network.loc[island_network.groupby(['island_id'])['sub_net_id'].apply(lambda x: x==min(x))]

In [40]:
# gdf of main network
df_final_network = nx.to_pandas_edgelist(sep_graph[0]).append(island_network[['source', 'target', 'link_id', 'highway', 'geometry', 'tunnel', 'name',
                                                                             'bridge', 'id', 'oneway', 'i_node', 'j_node', 'road_id']])
final_network = gpd.GeoDataFrame(df_final_network, geometry='geometry').set_crs(2285)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [41]:
final_network.head()


,source,target,link_id,highway,geometry,tunnel,name,bridge,id,oneway,i_node,j_node,road_id
0,1091875,984218,1,motorway_link,"LINESTRING (1276693.013 238146.063, 1276907.48...",None,None,viaduct,b-1,yes,1091875,984218,4.0
1,1091875,1055723,1287,motorway,"LINESTRING (1274523.918 237981.621, 1274675.07...",None,None,viaduct,b-2154,yes,1055723,1091875,116932.0
2,1091875,678437,1398,motorway,"LINESTRING (1276693.013 238146.063, 1277258.16...",None,Portage Bay Viaduct,viaduct,b-2254,yes,1091875,678437,118810.0
3,984218,793128,649966,motorway_link,"LINESTRING (1277418.550 238151.097, 1277503.79...",None,None,None,r-341464,yes,984218,793128,100905.0
4,1055723,970305,6990,motorway,"LINESTRING (1274295.965 237950.522, 1274523.91...",None,None,None,r-1,yes,970305,1055723,1.0


In [42]:
final_network.columns

Index(['source', 'target', 'link_id', 'highway', 'geometry', 'tunnel', 'name',
       'bridge', 'id', 'oneway', 'i_node', 'j_node', 'road_id'],
      dtype='object')

In [46]:
final_network2.crs

<Projected CRS: EPSG:2285>
Name: NAD83 / Washington North (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: USA - Washington - SPCS83 - N
- bounds: (-124.79, 47.08, -117.02, 49.05)
Coordinate Operation:
- name: SPCS83 Washington North zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [62]:
final_network.head()

,source,target,link_id,highway,geometry,tunnel,name,bridge,id,oneway,i_node,j_node,road_id
0,1091875,984218,1,motorway_link,"LINESTRING (1276693.013 238146.063, 1276907.48...",None,None,viaduct,b-1,yes,1091875,984218,4.0
1,1091875,1055723,1287,motorway,"LINESTRING (1274523.918 237981.621, 1274675.07...",None,None,viaduct,b-2154,yes,1055723,1091875,116932.0
2,1091875,678437,1398,motorway,"LINESTRING (1276693.013 238146.063, 1277258.16...",None,Portage Bay Viaduct,viaduct,b-2254,yes,1091875,678437,118810.0
3,984218,793128,649966,motorway_link,"LINESTRING (1277418.550 238151.097, 1277503.79...",None,None,None,r-341464,yes,984218,793128,100905.0
4,1055723,970305,6990,motorway,"LINESTRING (1274295.965 237950.522, 1274523.91...",None,None,None,r-1,yes,970305,1055723,1.0


In [35]:
island_network.to_file(r'C:/Joanne_PSRC/temp_data/island_network.shp')
final_network.to_file(r'C:/Joanne_PSRC/temp_data/final_network2.shp')

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


# save links and nodes to csv files

In [21]:
nodes.columns

,node_id,geometry
0,1,POINT (1343547.622 138752.081)
1,2,POINT (1280957.837 155260.215)
2,3,POINT (1076887.545 84688.734)
3,4,POINT (1273355.163 228830.169)
4,5,POINT (1200754.406 54392.210)
...,...,...
1098761,1098762,POINT (1303640.851 180465.992)
1098762,1098763,POINT (1112507.509 1615.626)
1098763,1098764,POINT (1289038.209 247637.045)
1098764,1098765,POINT (1325684.535 108883.470)


In [2]:
nodes2 = nodes.copy().set_crs(2285).to_crs(4326)
nodes2['x'] = nodes2['geometry'].apply(lambda x: x.coords[0][0])
nodes2['y'] = nodes2['geometry'].apply(lambda x: x.coords[0][1])
nodes_list = pd.DataFrame(nodes2.drop(columns='geometry').rename(columns={"node_id": "id"}))
nodes_list.head()


,id,x,y
0,1,-122.031462,47.374047
1,2,-122.285270,47.416342
2,3,-123.100770,47.209407
3,4,-122.321719,47.617599
4,5,-122.599775,47.135278


In [33]:

res = [int(i) for i in osm_df['i_node'].tolist()]
res

[1091875,
 1055723,
 1091875,
 984218,
 970305,
 678437,
 793128,
 975990,
 599196,
 465452,
 880960,
 444601,
 880960,
 913380,
 509886,
 465452,
 168453,
 637918,
 444601,
 676610,
 13881,
 13881,
 191550,
 971117,
 1011040,
 378207,
 119406,
 917201,
 130402,
 863932,
 865177,
 481504,
 865177,
 537102,
 386619,
 681168,
 105968,
 521235,
 62497,
 672716,
 521235,
 98867,
 98867,
 732412,
 917626,
 419555,
 772017,
 917626,
 620619,
 1059963,
 475604,
 488239,
 509886,
 475604,
 1043604,
 54501,
 891325,
 685751,
 685751,
 544272,
 696674,
 487966,
 1072962,
 1052970,
 382278,
 79279,
 227641,
 1023540,
 199593,
 229263,
 447506,
 405144,
 866284,
 843816,
 83745,
 37591,
 318589,
 677764,
 978069,
 671341,
 83745,
 367966,
 38038,
 751274,
 957737,
 957737,
 921112,
 844205,
 843966,
 139211,
 864471,
 768390,
 548633,
 864471,
 697665,
 465501,
 165635,
 201618,
 248405,
 231735,
 165635,
 1047659,
 370656,
 168781,
 443018,
 83969,
 582559,
 214248,
 726456,
 735478,
 378921,
 26

In [5]:
final_network2 = osm_df[['link_id', 'i_node', 'j_node','oneway','geometry']].copy()
final_network2['distance_miles'] = final_network2['geometry'].length / 5280
final_network2['distance_ft'] = final_network2['geometry'].length
final_network2

,link_id,i_node,j_node,oneway,geometry,distance_miles,distance_ft
0,1,1091875,984218,yes,"LINESTRING (1276693.013 238146.063, 1276907.48...",0.140338,740.987182
1,1287,1055723,1091875,yes,"LINESTRING (1274523.918 237981.621, 1274675.07...",0.413011,2180.696518
2,1398,1091875,678437,yes,"LINESTRING (1276693.013 238146.063, 1277258.16...",0.131481,694.220718
3,649966,984218,793128,yes,"LINESTRING (1277418.550 238151.097, 1277503.79...",0.049674,262.276381
4,6990,970305,1055723,yes,"LINESTRING (1274295.965 237950.522, 1274523.91...",0.043573,230.064076
...,...,...,...,...,...,...,...
1337858,635325,429497,176266,None,"LINESTRING (1298938.432 369129.805, 1298977.81...",0.015482,81.744377
1337859,635321,1065772,176266,None,"LINESTRING (1299003.394 369003.050, 1299007.58...",0.024914,131.545383
1337860,781247,1065772,1018215,None,"LINESTRING (1299003.394 369003.050, 1299003.08...",0.179496,947.736354
1337861,635320,824300,1065772,None,"LINESTRING (1298638.164 368706.530, 1298699.91...",0.095362,503.511331


In [9]:
links = final_network2[['i_node', 'j_node','distance_miles']].copy()
links['from'] = links['i_node']
links['to'] = links['j_node']
links['distance'] = links['distance_miles']
links = links[['from', 'to','distance']]
links

,from,to,distance
0,1091875,984218,0.140338
1,1055723,1091875,0.413011
2,1091875,678437,0.131481
3,984218,793128,0.049674
4,970305,1055723,0.043573
...,...,...,...
1337858,429497,176266,0.015482
1337859,1065772,176266,0.024914
1337860,1065772,1018215,0.179496
1337861,824300,1065772,0.095362


In [20]:
final_network2[final_network2['oneway']!='yes'][['i_node', 'j_node','distance_miles', 'oneway']]

,i_node,j_node,distance_miles,oneway
7,975990,880960,0.024570,reversible
8,599196,975990,0.113450,reversible
9,465452,975990,0.020823,reversible
10,880960,637918,0.074664,reversible
13,913380,599196,0.554183,reversible
...,...,...,...,...
1337858,429497,176266,0.015482,None
1337859,1065772,176266,0.024914,None
1337860,1065772,1018215,0.179496,None
1337861,824300,1065772,0.095362,None


In [12]:
final_network_oppo = final_network2[final_network2['oneway']!='yes'].copy()

links_oppo = final_network_oppo[['i_node', 'j_node','distance_miles']].copy()
links_oppo['from'] = links_oppo['j_node']
links_oppo['to'] = links_oppo['i_node']
links_oppo['distance'] = links_oppo['distance_miles']
links = links.append(links_oppo[['from', 'to','distance']].copy())
links

,from,to,distance
0,1091875,984218,0.140338
1,1055723,1091875,0.413011
2,1091875,678437,0.131481
3,984218,793128,0.049674
4,970305,1055723,0.043573
...,...,...,...
1337858,176266,429497,0.015482
1337859,176266,1065772,0.024914
1337860,1018215,1065772,0.179496
1337861,1065772,824300,0.095362


In [30]:
keep = int(links['from'].tolist() + links['to'].tolist())
keep

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [29]:
nodes['node_id'].tolist()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [37]:
res = [int(i) for i in links['from'].tolist() + links['to'].tolist()]

keep_nodes = nodes[nodes['node_id'].isin(res)]
len(keep_nodes)

1075674

# remove disconnected networks

In [8]:

from pathlib import Path

def rm_sub_network(gdf, islands, crs):
    x_graph = nx.Graph()
    graph = nx.from_pandas_edgelist(gdf, 'i_node', 'j_node',
                                    ['link_id', 'highway', 'name', 'bridge', 'tunnel', 'oneway', 'road_id',
                                     'all_link_i', 'i_node', 'j_node', 'geometry'], x_graph)
    # find all sub_networks
    sep_graph = [graph.subgraph(c).copy() for c in sorted(nx.connected_components(graph), key=len, reverse=True)]

    # combine all edges other than the main network
    df_list = []
    for i in range(1, len(sep_graph)):
        df = nx.to_pandas_edgelist(sep_graph[i])
        df_list.append(df)
    all_disconnected = pd.concat(df_list, ignore_index=True)

    # create a gdf out of the disconnected network and clip with island shp
    gdf_disconnect = gpd.GeoDataFrame(all_disconnected, geometry='geometry').set_crs(crs)

    # keep only networks on islands: clip disconnected networks using islands
    # islands = gpd.read_file(Path(islands)).to_crs(crs)
    island_network = gpd.clip(gdf_disconnect, islands.to_crs(crs))

    # gdf of main network
    df_final_network = nx.to_pandas_edgelist(sep_graph[0]).append(island_network)
    final_network = gpd.GeoDataFrame(df_final_network, geometry='geometry').set_crs(crs)

    return final_network

In [10]:
main_network = rm_sub_network(osm_df, islands, 2258)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [13]:
osm_df.columns

Index(['link_id', 'highway', 'name', 'bridge', 'tunnel', 'oneway', 'road_id',
       'all_link_i', 'i_node', 'j_node', 'geometry'],
      dtype='object')

In [14]:
# only keep connected network.
x = nx.Graph()
G = nx.from_pandas_edgelist(osm_df, 'i_node', 'j_node', ['link_id', 'highway', 'name', 'bridge', 'tunnel', 'oneway', 'road_id',
                                                         'all_link_i', 'i_node', 'j_node', 'geometry'], x)

In [4]:
S = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)]

In [30]:
# save disconnected network to shp file
for i in range(len(S)):
    df = nx.to_pandas_edgelist(S[i])
    connected_links = osm_df[osm_df['all_link_i'].isin(df['all_link_i'])]
    connected_links.to_file(r'C:/Joanne_PSRC/temp_data/connected_links_%s.shp' % (str(i)))


C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [18]:
# combine all edges other than the main network
df_list = []

for i in range(1,len(S)):
    df = nx.to_pandas_edgelist(S[i])
    df_list.append(df)

all_disconnected = pd.concat(df_list,ignore_index=True)
# create a gdf out of the disconnected network and clip with island shp
gdf_disconnect = gpd.GeoDataFrame(all_disconnected, geometry='geometry').set_crs(2285)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [3]:

# island_network = gpd.clip(gdf_disconnect, islands)

In [30]:
# gdf of main network
# final_network = osm_df[osm_df['all_link_i'].isin(nx.to_pandas_edgelist(S[0])['all_link_i'] + island_network['all_link_i'])]
final_network = nx.to_pandas_edgelist(S[0]).append(island_network)

C:\Users\JLin\Anaconda3\envs\geomap\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [33]:
final_network1 = gpd.GeoDataFrame(final_network, geometry='geometry').set_crs(2285)

In [34]:
final_network1.to_file(r'C:/Joanne_PSRC/temp_data/connect_network.shp')